<a href="https://colab.research.google.com/github/sherna90/inteligencia_artificial/blob/master/7.-cnn_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet                                                                                                                    # IGNORE_COPYRIGHT: cleared by OSS licensing
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Transfer learning and fine-tuning

In this tutorial, you will learn how to classify images of cats and dogs by using transfer learning from a pre-trained network.

A pre-trained model is a saved network that was previously trained on a large dataset, typically on a large-scale image-classification task. You either use the pretrained model as is or use transfer learning to customize this model to a given task.

The intuition behind transfer learning for image classification is that if a model is trained on a large and general enough dataset, this model will effectively serve as a generic model of the visual world. You can then take advantage of these learned feature maps without having to start from scratch by training a large model on a large dataset.

In this notebook, you will try two ways to customize a pretrained model:

1. Feature Extraction: Use the representations learned by a previous network to extract meaningful features from new samples. You simply add a new classifier, which will be trained from scratch, on top of the pretrained model so that you can repurpose the feature maps learned previously for the dataset.

 You do not need to (re)train the entire model. The base convolutional network already contains features that are generically useful for classifying pictures. However, the final, classification part of the pretrained model is specific to the original classification task, and subsequently specific to the set of classes on which the model was trained.

1. Fine-Tuning: Unfreeze a few of the top layers of a frozen model base and jointly train both the newly-added classifier layers and the last layers of the base model. This allows us to "fine-tune" the higher-order feature representations in the base model in order to make them more relevant for the specific task.

You will follow the general machine learning workflow.

1. Examine and understand the data
1. Build an input pipeline, in this case using Keras ImageDataGenerator
1. Compose the model
   * Load in the pretrained base model (and pretrained weights)
   * Stack the classification layers on top
1. Train the model
1. Evaluate model


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

## Data preprocessing

### Data download

In this tutorial, you will use a dataset containing several thousand images of cats and dogs. Download and extract a zip file containing the images, then create a `tf.data.Dataset` for training and validation using the `tf.keras.utils.image_dataset_from_directory` utility. You can learn more about loading images in this [tutorial](https://www.tensorflow.org/tutorials/load_data/images).

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ruta al directorio principal que contiene subdirectorios para cada raza de perro
dataset_path = 'D:/Users/Asus/Desktop/informeia/datasets/images/Images'

# Rutas para el directorio de entrenamiento y el directorio de validación
train_dir = os.path.join(dataset_path, 'train')
validation_dir = os.path.join(dataset_path, 'validation')

# Tamaño de las imágenes y tamaño del lote
image_size = (100, 100)
batch_size = 16

# Configuración del generador de imágenes para preprocesamiento
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# Generador de datos para el conjunto de entrenamiento
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    classes=['n02088364-beagle', 'n02099601-golden_retriever'],
    class_mode='categorical',
    shuffle=True
)

# Configuración del generador de imágenes para preprocesamiento en validación (sin aumentos)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Generador de datos para el conjunto de validación
validation_dataset = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    classes=['n02088364-beagle', 'n02099601-golden_retriever'],
    class_mode='categorical',
    shuffle=False
)

# Visualizar información de los generadores
print(train_dataset.class_indices)
print(train_dataset.filenames)


In [ ]:
#validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
#                                                                 shuffle=True,
#                                                                 batch_size=BATCH_SIZE,
#                                                                 image_size=IMG_SIZE)

Show the first nine images and labels from the training set:

In [ ]:
import matplotlib.pyplot as plt

# Obtén los nombres de las clases
class_names = list(train_dataset.class_indices.keys())

# Crea una nueva figura de tamaño 10x10 pulgadas
plt.figure(figsize=(10, 10))

# Obtén las imágenes y etiquetas del conjunto de entrenamiento
images, labels = next(train_dataset)

# Itera sobre las primeras 9 imágenes del lote
for i in range(9):
    # Crea un subgráfico en una cuadrícula de 3x3
    ax = plt.subplot(3, 3, i + 1)
    
    # Muestra la forma de la imagen
    print("Shape of the image:", images[i].shape)
    
    # Convierte la imagen a un array NumPy y ajusta el tipo de datos
    image_to_show = (images[i] * 255).astype("uint8")
    
    # Muestra la imagen
    plt.imshow(image_to_show)

    # Establece el título del subgráfico como el nombre de la clase
    plt.title(class_names[tf.argmax(labels[i]).numpy()])

    # Desactiva los ejes para una presentación más limpia
    plt.axis("off")

# Muestra la figura
plt.show()



In [ ]:
import tensorflow as tf

# Tamaño del lote
batch_size = 16

# Obtén un generador de imágenes para el conjunto de datos de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'  # Puedes ajustar esto según tus necesidades
)

# Inicializa una lista para almacenar los lotes
ds_train_batches = []

# Carga el conjunto de datos por lotes
for _ in range(len(train_generator)):
    batch = next(train_generator)
    ds_train_batches.append(batch)

# Ahora, ds_train_batches es una lista que contiene todos los lotes del conjunto de datos de entrenamiento


In [ ]:
# Dividir el conjunto de datos en entrenamiento y validación
train_size = int(0.8 * len(ds_train_batches))
val_size = len(ds_train_batches) - train_size

# Conjunto de datos de entrenamiento
ds_train = ds_train_batches[:train_size]

# Conjunto de datos de validación
ds_val = ds_train_batches[train_size:]


In [ ]:
# Supongamos que tienes un generador de imágenes como train_generator
# y que las etiquetas están en el formato (imagenes, etiquetas)

# Obtener un batch de imágenes y etiquetas
images, labels = next(train_generator)

# Asegurarse de que las etiquetas sean un array NumPy unidimensional de enteros
labels = np.array(labels).flatten().astype(int)

# Calcular la cantidad de ejemplos positivos y negativos
neg, pos = np.bincount(labels)

# Calcular el total de ejemplos
total = neg + pos

# Imprimir información sobre la distribución de clases
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))


As the original dataset doesn't contain a test set, you will create one. To do so, determine how many batches of data are available in the validation set using `tf.data.experimental.cardinality`, then move 20% of them to a test set.

In [ ]:
# Obtener la cantidad de lotes en el conjunto de validación
val_batches = len(ds_val)

# Calcular cuántos lotes corresponden al 20% para el conjunto de prueba
test_batches = val_batches // 5

# Crear el conjunto de datos de prueba tomando el 20% de los lotes del conjunto de validación
ds_test = ds_val[:test_batches]

# Actualizar el conjunto de validación para excluir los lotes utilizados para prueba
ds_val = ds_val[test_batches:]

print(len(ds_val))

In [ ]:
# Imprimir la cantidad de lotes en el conjunto de validación
print('Number of validation batches: %d' % len(ds_val))

# Imprimir la cantidad de lotes en el conjunto de prueba
print('Number of test batches: %d' % len(ds_test))


### Configure the dataset for performance

Use buffered prefetching to load images from disk without having I/O become blocking. To learn more about this method see the [data performance](https://www.tensorflow.org/guide/data_performance) guide.

In [ ]:
# Crear conjuntos de datos a partir de generadores
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, image_size[0], image_size[1], 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2), dtype=tf.float32)  # Ajusta la forma según tus necesidades
    )
)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, image_size[0], image_size[1], 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2), dtype=tf.float32)  # Ajusta la forma según tus necesidades
    )
)

# Ajustar la forma de las etiquetas en los conjuntos de datos
train_dataset = train_dataset.map(lambda img, label: (img, label[:, 0]), num_parallel_calls=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.map(lambda img, label: (img, label[:, 0]), num_parallel_calls=tf.data.AUTOTUNE)

# Configurar prefetching para los conjuntos de datos
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)



In [ ]:
# Calcular el número total de muestras en los conjuntos de entrenamiento, prueba y validación
num_train = len(ds_train) * batch_size
num_val = len(ds_val) * batch_size
num_test = len(ds_test) * batch_size

# Crear un gráfico de barras para visualizar la distribución de las muestras
fig, ax = plt.subplots()

datasets_cardinality = ['train', 'test', 'validation']
counts = [num_train, num_test, num_val]

ax.bar(datasets_cardinality, counts)
ax.legend(title='Number of Samples')
plt.show()


### Use data augmentation

When you don't have a large image dataset, it's a good practice to artificially introduce sample diversity by applying random, yet realistic, transformations to the training images, such as rotation and horizontal flipping. This helps expose the model to different aspects of the training data and reduce [overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit). You can learn more about data augmentation in this [tutorial](https://www.tensorflow.org/tutorials/images/data_augmentation).

In [ ]:
#data_augmentation = tf.keras.Sequential([
#  tf.keras.layers.RandomFlip('horizontal'),
#  tf.keras.layers.RandomRotation(0.2),
#])

Note: These layers are active only during training, when you call `Model.fit`. They are inactive when the model is used in inference mode in `Model.evaluate`, `Model.predict`, or `Model.call`.

Let's repeatedly apply these layers to the same image and see the result.

In [ ]:
# Crear una secuencia de aumento de datos
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2)
])

# Obtener un lote de imágenes del conjunto de entrenamiento
for images, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    
    # Seleccionar la primera imagen del lote
    first_image = images[0]
    
    # Aplicar aumento de datos y visualizar las imágenes aumentadas
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0].numpy())
        plt.axis('off')

# Mostrar la figura
plt.show()


In [ ]:
pre_process_data = True  # Cambiar a True si deseas aplicar el aumento de datos

if pre_process_data:
    # Aplicar la secuencia de aumento de datos al conjunto de entrenamiento
    train_dataset = train_dataset.map(
        lambda img, label: (data_augmentation(img), label),
        num_parallel_calls=tf.data.AUTOTUNE
    )


# Small CNN

Ahora volvemos al español y creamos una red convolucional "from scratch"

In [ ]:
# Importar módulos necesarios
from tensorflow.keras import datasets, layers, models

IMG_SIZE = (100, 100)

# Crear un modelo secuencial
baseline_model = models.Sequential()

# Normalizar los valores de píxeles de las imágenes en el rango [0, 1]
baseline_model.add(layers.Rescaling(1.0 / 255))

# Primera capa convolucional con 32 filtros de 3x3, activación ReLU y entrada de imágenes
baseline_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))

# Capa de reducción de muestreo (MaxPooling) para reducir el tamaño espacial
baseline_model.add(layers.MaxPooling2D((2, 2)))

# Segunda capa convolucional con 64 filtros de 3x3 y activación ReLU
#baseline_model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Otra capa de reducción de muestreo para seguir reduciendo el tamaño espacial
baseline_model.add(layers.MaxPooling2D((2, 2)))

# Tercera capa convolucional con 64 filtros de 3x3 y activación ReLU
#baseline_model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Aplanar la salida para prepararla para capas densas (totalmente conectadas)
baseline_model.add(layers.Flatten())

# Capa densa (totalmente conectada) con 64 unidades y activación ReLU
#baseline_model.add(layers.Dense(64, activation='relu'))

# Capa de salida con 1 unidad (para clasificación binaria) y sin activación
baseline_model.add(layers.Dense(1, activation=None))


In [ ]:
from tensorflow.keras.activations import sigmoid

predictions = baseline_model(images)


In [ ]:
plt.imshow((images[3].numpy() * 255).astype("uint8"))




In [ ]:
# Calcular la puntuación (score) usando la función sigmoid
score = float(sigmoid(predictions[3][0]))

# Obtener la imagen del conjunto de datos y convertirla a un array NumPy
image_to_show = (images[3].numpy() * 255).astype("uint8")

# Mostrar la imagen
plt.imshow(image_to_show)

# Establecer la etiqueta de predicción basada en el umbral de 0.5
pred_label = score > 0.5

# Establecer el título del gráfico con la etiqueta de predicción
plt.title(class_names[pred_label])

# Desactivar los ejes para una presentación más limpia
plt.axis('off')

# Mostrar el gráfico
plt.show()


In [ ]:
baseline_model.summary()

In [ ]:
print(tf.config.list_physical_devices('GPU'))


In [ ]:
reduced_train_dataset = train_dataset.take(2)  # Tomar solo los primeros 100 ejemplos
reduced_val_dataset = validation_dataset.take(1)  # Tomar solo los primeros 20 ejemplos


In [ ]:
# Compilar el modelo
baseline_model.compile(optimizer='sgd',
                       loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                       metrics=['accuracy'])

# Número de épocas para entrenar el modelo
num_epochs = 1

# Entrenar el modelo usando el conjunto de entrenamiento y validar en el conjunto de prueba
history = baseline_model.fit(train_dataset, epochs=num_epochs, validation_data=validation_dataset)


HASTA AQUI FUNCIONA DE PANA Y SE QUEDA ENTRENANDO EL MODEDLO INFINITAMENTE, DICE QUE CON GRAFICA EN VOLA SE PUEDE ACELERAR.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
epochs=range(num_epochs)
fig.suptitle('Accuracy / Loss')
#fig.subplots_adjust(hspace=10)
ax1.plot(epochs,history.history['accuracy'], label='accuracy')
ax1.plot(epochs,history.history['val_accuracy'], label = 'val_accuracy')
ax1.set_ylabel('Accuracy')
ax2.plot(epochs,history.history['loss'], label='loss')
ax2.plot(epochs,history.history['val_loss'], label = 'val_loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epochs')



In [ ]:
test_loss, test_acc = baseline_model.evaluate(validation_dataset, verbose=0)


In [ ]:
print('Train Loss :  {:0.4f}, Train Accuracy  : {:0.4f} '.format(history.history['loss'][-1], history.history['accuracy'][-1]))
print('Val Loss   :  {:0.4f}, Val Accuracy    : {:0.4f} '.format(history.history['val_loss'][-1], history.history['val_accuracy'][-1]))
print('Test Loss  :  {:0.4f}, Test Accuracy   : {:0.4f} '.format(test_loss, test_acc))


In [ ]:
predictions=baseline_model.predict(images)


In [ ]:
plt.imshow(images[2] / 255)
pred_label=int((predictions[2]>0.5))
plt.title(class_names[pred_label])
plt.axis('off')

# Transfer Learning

Descargamos modelos pre-entrenados desde:

https://keras.io/api/applications/

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(include_top=False, weights='imagenet',input_shape=(160,160,3))

In [ ]:
predictions=base_model.predict(images)
plt.imshow(images[2] / 255)
plt.axis('off')

In [ ]:
predictions.shape

In [ ]:
base_model.summary()

This feature extractor converts each `160x160x3` image into a `5x5x1280` block of features. Let's see what it does to an example batch of images:

## Feature extraction
In this step, you will freeze the convolutional base created from the previous step and to use as a feature extractor. Additionally, you add a classifier on top of it and train the top-level classifier.

### Freeze the convolutional base

It is important to freeze the convolutional base before you compile and train the model. Freezing (by setting layer.trainable = False) prevents the weights in a given layer from being updated during training. MobileNet V2 has many layers, so setting the entire model's `trainable` flag to False will freeze all of them.

In [ ]:
base_model.trainable = False

### Important note about BatchNormalization layers

Many models contain `tf.keras.layers.BatchNormalization` layers. This layer is a special case and precautions should be taken in the context of fine-tuning, as shown later in this tutorial.

When you set `layer.trainable = False`, the `BatchNormalization` layer will run in inference mode, and will not update its mean and variance statistics.

When you unfreeze a model that contains BatchNormalization layers in order to do fine-tuning, you should keep the BatchNormalization layers in inference mode by passing `training = False` when calling the base model. Otherwise, the updates applied to the non-trainable weights will destroy what the model has learned.

For more details, see the [Transfer learning guide](https://www.tensorflow.org/guide/keras/transfer_learning).

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

### Add a classification head

To generate predictions from the block of features, average over the spatial `5x5` spatial locations, using a `tf.keras.layers.GlobalAveragePooling2D` layer to convert the features to  a single 1280-element vector per image.

In [ ]:
def build_model(base_model,num_classes):
    inputs = layers.Input(shape=(160, 160, 3))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation=None, name="pred")(x)
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    return model

Apply a `tf.keras.layers.Dense` layer to convert these features into a single prediction per image. You don't need an activation function here because this prediction will be treated as a `logit`, or a raw prediction value. Positive numbers predict class 1, negative numbers predict class 0.

In [ ]:
fine_tune_model=build_model(base_model,1)

In [ ]:
predictions=fine_tune_model(images)

In [ ]:
predictions.shape

Build a model by chaining together the data augmentation, rescaling, `base_model` and feature extractor layers using the [Keras Functional API](https://www.tensorflow.org/guide/keras/functional). As previously mentioned, use `training=False` as our model contains a `BatchNormalization` layer.

In [ ]:
fine_tune_model.summary()

### Compile the model

Compile the model before training it. Since there are two classes, use the `tf.keras.losses.BinaryCrossentropy` loss with `from_logits=True` since the model provides a linear output.

In [ ]:
fine_tune_model.compile(optimizer='sgd',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = fine_tune_model.fit(train_dataset, epochs=30, validation_data=test_dataset)

### Train the model

After training for 10 epochs, you should see ~96% accuracy on the validation set.


In [ ]:

loss0, accuracy0 = fine_tune_model.evaluate(validation_dataset)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
fig.suptitle('Accuracy / Loss')
ax1.plot(history.history['accuracy'], label='accuracy')
ax1.plot(history.history['val_accuracy'], label = 'val_accuracy')
ax1.set_ylabel('Accuracy')
ax2.plot(history.history['loss'], label='loss')
ax2.plot(history.history['val_loss'], label = 'val_loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epochs')


## Summary

* **Using a pre-trained model for feature extraction**:  When working with a small dataset, it is a common practice to take advantage of features learned by a model trained on a larger dataset in the same domain. This is done by instantiating the pre-trained model and adding a fully-connected classifier on top. The pre-trained model is "frozen" and only the weights of the classifier get updated during training.
In this case, the convolutional base extracted all the features associated with each image and you just trained a classifier that determines the image class given that set of extracted features.

* **Fine-tuning a pre-trained model**: To further improve performance, one might want to repurpose the top-level layers of the pre-trained models to the new dataset via fine-tuning.
In this case, you tuned your weights such that your model learned high-level features specific to the dataset. This technique is usually recommended when the training dataset is large and very similar to the original dataset that the pre-trained model was trained on.

To learn more, visit the [Transfer learning guide](https://www.tensorflow.org/guide/keras/transfer_learning).
